In [1]:
import awswrangler as wr
from datetime import datetime, timedelta
import os
import sys

In [2]:
repo_name = "mercury-pipeline-metaapi-sam"
sys.path.append(os.getcwd().split(repo_name)[0] + repo_name + "/workflow") 

In [3]:
from src.config import *
from src.io import *
from src.utils import *

In [4]:
from app import *

In [5]:
# result = lambda_handler({"function": "setup_save_raw_ticks", "stop":False}, None)
# result

In [6]:
# await process_batch_async(save_raw_ticks, batch_id=3)

In [7]:
# lambda_handler({"function": "save_raw_ticks", "batch_id":result[0]}, None)

In [8]:
# result = lambda_handler({"function": "setup_save_clean_ticks"},  None)
# result

In [9]:
# lambda_handler({"function": "save_clean_ticks", "batch_id":result[0]}, None)

In [10]:
# result = lambda_handler({"function": "setup_save_candles_from_ticks"},  None)
# result

In [11]:
# lambda_handler({"function": "save_candles_from_ticks", "batch_id":result[0]},  None)

In [12]:
result = lambda_handler(
    {
        "function": "setup_save_candles_transformed",
        "name_dataset_in": "from_ticks",
        "name_dataset_out": "ffill",
        "function_names": ["ffill"],
    },
    None
)
max(result)

[350, 351, 352, 353, 354, 355, 356, 357]

In [13]:
# result

In [15]:
%%time
lambda_handler({"function": "save_candles_transformed", "batch_id":result[0][0]},  None)

AssertionError: 

In [ ]:
%%time
df = wr.s3.read_parquet(
    f"s3://{BUCKET}/{DIR_CANDLES_ROOT}/ffill/",
    dataset=True,
    partition_filter = lambda x: True if all([
        x["symbol"] == "GBPUSD", x["frequency"] == "15T"
    ]) else False,
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.step(df["timestamp_utc.open"], df["ask.close"])

In [16]:
f_filter = lambda x: True if all([
    x["symbol"] == "GBPUSD",
    x["frequency"] == "15T",
]) else False

In [17]:
keys = [
    x for x in wr.s3.list_objects(
        f"s3://{BUCKET}/{DIR_CANDLES_ROOT}/ffill/",
        suffix=".parquet"
    ) if f_filter(get_parameters_from_key(x))
]

In [18]:
len(keys)

3228

In [ ]:
wr.s3.download(

In [ ]:
%%time
dfs = [wr.s3.read_parquet(x) for x in keys]

In [ ]:
%%time
wr.s3.read_parquet(keys)

In [20]:
%%time
for key in keys:
    wr.s3.download(
        key,
        f"./tmp/{key.split('candles/')[-1]}"
    )

FileNotFoundError: [Errno 2] No such file or directory: './tmp/ffill/symbol=GBPUSD/frequency=15T/year=2021/month=12/date=20211224/hour=01/9234889da4804a4297e657c9a7a101c6.snappy.parquet'

SyntaxError: invalid syntax (2383361945.py, line 1)